In [1]:
import pandas as pd
import numpy as np
from tqdm.notebook import tqdm
from copy import deepcopy
from xgboost import XGBRegressor
from sklearn.model_selection import RandomizedSearchCV
from sklearn.pipeline import Pipeline

from my_splitter import MySplitter
from my_transformer import MyTransformer
from recursive_pipeline import RecursivePipeline

tqdm.pandas()

ID_SIZE = 20

In [12]:
import os

if os.getcwd() != "/root/restaurants":
    os.chdir("..")
os.getcwd()

'/root/restaurants'

In [13]:
store_info = pd.read_csv("data/raw/air_store_info.csv")
store_info = store_info.rename(
    columns={
        "air_store_id": "store_id",
        "air_genre_name": "genre_name",
        "air_area_name": "area_name",
    }
)

date_info = pd.read_csv("data/raw/date_info.csv")
date_info = date_info.rename(columns={"calendar_date": "date"})
date_info["date"] = date_info["date"].astype("string")

In [14]:
submission = pd.read_csv("data/raw/sample_submission.csv")
data = pd.read_csv("data/processed/data.csv")
data["date"] = pd.to_datetime(data["date"])
data.head()

new_submission = deepcopy(submission)
new_submission["store_id"] = new_submission["id"].str[:ID_SIZE]
new_submission["date"] = new_submission["id"].str[ID_SIZE + 1 :]

new_submission = pd.merge(new_submission, date_info, on="date")
new_submission = pd.merge(new_submission, store_info, on="store_id")

new_submission["date"] = pd.to_datetime(new_submission["date"])
new_submission["year"] = new_submission["date"].dt.year
new_submission["month"] = new_submission["date"].dt.month
new_submission["day"] = new_submission["date"].dt.day

In [15]:
data = data[
    [
        "store_id",
        "date",
        "day_of_week",
        "holiday_flg",
        "genre_name",
        "area_name",
        "latitude",
        "longitude",
        "year",
        "month",
        "day",
        "visitors",
    ]
]

In [16]:
import pandas as pd
from tsfresh import extract_features, extract_relevant_features
from tsfresh.utilities.dataframe_functions import make_forecasting_frame
from tsfresh import select_features


# Extract features
extracted_features = extract_features(data.drop(columns=["visitors"]).head(100), column_id='store_id', column_sort='date')

y = data["visistors"].head(100)
selected_features = select_features(extracted_features, y)

print(selected_features)


Feature Extraction:   0%|          | 0/20 [00:00<?, ?it/s]


TypeError: unsupported operand type(s) for /: 'str' and 'int'

In [ ]:
pipeline = RecursivePipeline(
    pipeline=Pipeline(
        steps=[
            ("transformer", MyTransformer()),
            (
                "model",
                XGBRegressor(
                    objective="reg:squaredlogerror",
                    random_state=42,
                    enable_categorical=True,
                ),
            ),
        ]
    )
)

X = data.drop(columns=["visitors"]).reset_index(drop=True)[:100]
X_pred = data.drop(columns=["visitors"]).reset_index(drop=True)[100:130]
y = data["visitors"].reset_index(drop=True)[:100]

pipeline.fit(X, y)
pipeline.predict(X_pred)

  0%|          | 0/10 [00:00<?, ?it/s]

array([10.829223, 10.829223, 10.829223, 10.532826, 10.532826, 10.532826,
       10.532826, 10.829223, 10.829223, 10.829223, 10.829223, 10.829223,
       10.532826, 10.532826, 10.829223, 10.532826, 10.829223, 10.829223,
       10.829223, 10.829223, 10.532826, 10.829223, 10.532826, 10.532826,
       10.829223, 10.829223, 10.829223, 10.829223, 10.829223, 10.829223],
      dtype=float32)

In [ ]:
pipeline = RecursivePipeline(
    pipeline=Pipeline(
        steps=[
            ("transformer", MyTransformer()),
            (
                "model",
                XGBRegressor(
                    objective="reg:squaredlogerror",
                    random_state=42,
                    enable_categorical=True,
                ),
            ),
        ]
    )
)


param_grid = {
    "pipeline__model__n_estimators": [5, 10, 20, 30, 40, 50, 70],
    "pipeline__model__learning_rate": [0.001, 0.01, 0.03, 0.05, 0.1, 0.3, 0.5],
    "pipeline__model__max_depth": np.arange(2, 20, 1),
}

validation_size = new_submission["date"].nunique()
cv = MySplitter(test_size=validation_size)
rscv = RandomizedSearchCV(
    estimator=pipeline,
    cv=cv,
    param_distributions=param_grid,
    scoring="neg_root_mean_squared_log_error",
    n_jobs=2,
    verbose=5,
    n_iter=10,
)


X = data.drop(columns=["visitors"]).reset_index(drop=True)
y = data["visitors"].reset_index(drop=True)

rscv.fit(X, y)

Fitting 5 folds for each of 10 candidates, totalling 50 fits


KeyboardInterrupt: 